In [ ]:
!nvidia-smi

Sat Jun 26 08:13:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 99% 1.03G/1.04G [00:12<00:00, 127MB/s]
100% 1.04G/1.04G [00:12<00:00, 88.5MB/s]


In [ ]:
# get the code form github
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 547, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 547 (delta 333), reused 505 (delta 291), pack-reused 0
Receiving objects: 100% (547/547), 38.27 MiB | 23.52 MiB/s, done.
Resolving deltas: 100% (333/333), done.


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

In [ ]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 30  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
pretrained_embeddings = False
fine_tune_embeddings = False
checkpoint = None  # path to checkpoint, None if none


In [ ]:
DATA_NAME = 'flickr8k_ar'

# local
# DATA_JSON_PATH = 'ar_data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/ar_data.json'
IMGS_PATH = 'flickr8k/images/'

In [ ]:
max_seq = 30
vocab = build_vocab(DATA_JSON_PATH, max_seq=max_seq)
vocab_len = len(vocab); vocab_len

100%|██████████| 24000/24000 [00:00<00:00, 380040.76it/s]


5788

In [ ]:
list(vocab.itos.keys())[:10], list(vocab.itos.values())[:10]

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 ['<pad>',
  '<sos>',
  '<eos>',
  '<unk>',
  'طفلة',
  'صغيرة',
  'تتسلق',
  'إلى',
  'كلب',
  'أسود'])

In [ ]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

logger_dic = {
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'max_seq_length': max_seq,
    'vocab_size': vocab_len,
    'enocder': 'resnet101',
    'dropout': dropout,
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim 
    
}

In [ ]:
t_params

{'batch_size': 256,
 'data_name': 'flickr8k_ar',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/ar_data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7f7e18aaefd0>,
 'workers': 2}

In [ ]:
# experiment name
name = DATA_NAME + ""
# path
log_dir = '/content/drive/MyDrive/ImageCaptioning/flickr8_ar/experiments'

logger = SummaryWriter(log_dir=osp.join(log_dir, name))

In [18]:
fit(t_params=t_params, m_params=m_params, logger=logger)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth



Loading Data
Dataset split: train
Unique images: 6000
Total size: 18000
Dataset split: val
Unique images: 1000
Total size: 3000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/71]	Batch Time 8.471 (8.471)	Data Load Time 3.622 (3.622)	Loss 9.6074 (9.6074)	Top-5 Accuracy 0.144 (0.144)
Epoch train time 154.997 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/12]	Batch Time 5.519 (5.519)	Loss 6.4465 (6.4465)	Top-5 Accuracy 36.128 (36.128)	
----- Bleu-n Scores -----
1: 34.28116163569975
2: 15.004966110980616
3: 5.752228240109723
4: 2.3468998100455627
-------------------------

 * LOSS - 6.468, TOP-5 ACCURACY - 35.838, BLEU-4 - 2.3468998100455627

Epoch validation time 32.159 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [1][0/71]	Batch Time 5.749 (5.749)	Data Load Time 3.375 (3.375)	Loss 5.9978 (5.9978)	Top-5 Accuracy 40.631 (40.631)
Epoch train time 154.220 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/12]	Batch Time 5.638 (5.638)	Loss 6.0548 (6.0548)	Top-5 Accuracy 42.096 (42.096)	
--

In [19]:
m = load_checkpoint("/content/BEST_checkpoint_flickr8k_ar.pth.tar")

Loaded Checkpoint!!
Last Epoch: 8
Best Bleu-4: 5.715176451490768


In [20]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_ar.pth.tar'
# epochs = 30

t_params['batch_size'] = batch_size
t_params['data_name'] = t_params['data_name'] + "_finetune" 
t_params['fine_tune_encoder'] = True
t_params['decoder_lr'] = t_params['decoder_lr'] / 10
# t_params['epochs'] = epochs
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_ar_finetune',
 'decoder_lr': 4e-05,
 'df_path': 'Image-Captioning/ar_data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7f7e18aaefd0>,
 'workers': 2}

In [21]:
fit(t_params, checkpoint=checkpoint, m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 9
Loading Data
Dataset split: train
Unique images: 6000
Total size: 18000
Dataset split: val
Unique images: 1000
Total size: 3000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [9][0/282]	Batch Time 4.199 (4.199)	Data Load Time 0.960 (0.960)	Loss 4.9865 (4.9865)	Top-5 Accuracy 52.319 (52.319)
Epoch: [9][100/282]	Batch Time 1.235 (1.233)	Data Load Time 0.001 (0.010)	Loss 5.1665 (4.9120)	Top-5 Accuracy 49.912 (52.423)
Epoch: [9][200/282]	Batch Time 1.229 (1.217)	Data Load Time 0.001 (0.006)	Loss 4.8351 (4.8706)	Top-5 Accuracy 53.012 (53.206)
Epoch train time 341.644 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/47]	Batch Time 1.474 (1.474)	Loss 5.5957 (5.5957)	Top-5 Accuracy 45.269 (45.269)	
----- Bleu-n Scores -----
1: 43.30506294300831
2: 22.758277

In [22]:
!cp BEST_checkpoint_flickr8k_ar_finetune.pth.tar /content/drive/MyDrive/ImageCaptioning/flickr8_ar

In [23]:
checkpoint = load_checkpoint("BEST_checkpoint_flickr8k_ar_finetune.pth.tar")
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval();

Loaded Checkpoint!!
Last Epoch: 19
Best Bleu-4: 6.862300456763069


In [24]:
from eval import test_score

test_dict = {}

for i in [1, 3, 5]:
    
    b1, b2, b3, b4 = test_score(i, encoder, decoder, IMGS_PATH, DATA_JSON_PATH, vocab)
    if i == 3:
        test_dict['b1'] = b1
        test_dict['b2'] = b2
        test_dict['b3'] = b3
    
    test_dict[f'b4-b{i}'] = b4

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 1:   0%|          | 0/3000 [00:00<?, ?it/s]

Dataset split: test
Unique images: 1000
Total size: 3000


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
EVALUATING AT BEAM SIZE 1: 100%|██████████| 3000/3000 [01:26<00:00, 34.86it/s]


----- Bleu-n Scores -----
1: 39.95516959427624
2: 25.77931292578018
3: 15.10781606368901
4: 8.64667118061901
-------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 3:   0%|          | 0/3000 [00:00<?, ?it/s]

...

----- Bleu-n Scores -----


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 5:   0%|          | 0/3000 [00:00<?, ?it/s]

1: 40.373911854332825
2: 26.615634981477243
3: 16.09711443584611
4: 9.384744789396482
-------------------------
Dataset split: test
Unique images: 1000
Total size: 3000


EVALUATING AT BEAM SIZE 5: 100%|██████████| 3000/3000 [01:43<00:00, 28.92it/s]


----- Bleu-n Scores -----
1: 39.84779810028518
2: 26.47471808239697
3: 15.929967050767372
4: 9.330062157193685
-------------------------


In [25]:
test_dict

{'b1': 40.373911854332825,
 'b2': 26.615634981477243,
 'b3': 16.09711443584611,
 'b4-b1': 8.64667118061901,
 'b4-b3': 9.384744789396482,
 'b4-b5': 9.330062157193685}

In [26]:
# final results -> different from training and validation scalars
results_dic =  {
    # train & valid
    'total_epochs': 5.653,
    'b-1/test': test_dict['b1'],
    'b-2/test': test_dict['b2'],
    'b-3/test': test_dict['b3'],
    'b-4/b3': test_dict['b4-b3'],
    'b-4/b1': test_dict['b4-b1'],
    'b-4/b5': test_dict['b4-b5']
}

In [27]:
logger.add_hparams(logger_dic, results_dic, run_name='Arabic_30maxlen')